In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, RepeatVector, TimeDistributed, Concatenate, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import math

In [ ]:
# Function to create sequences for time series prediction
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out + 1):
        X.append(data[i:i + n_steps_in])
        y.append(data[i + n_steps_in:i + n_steps_in + n_steps_out])
    return np.array(X), np.array(y)

# Add new function to create cyclical time features
def add_cyclical_features(df):
    """
    Add cyclical encoding of time features to capture seasonality
    """
    # For month: convert to sine and cosine components
    df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

    # For day of month
    days_in_month = 30  # Approximation
    df['day_sin'] = np.sin(2 * np.pi * df['Day']/days_in_month)
    df['day_cos'] = np.cos(2 * np.pi * df['Day']/days_in_month)

    return df

# 1. Add this new function for circular wind direction encoding
def encode_wind_direction(df):
    """
    Convert wind direction from degrees to sine and cosine components
    to properly handle its circular nature
    """
    if 'Wind_Direction' in df.columns:
        # Convert degrees to radians and then to sine/cosine components
        df['wind_dir_sin'] = np.sin(np.radians(df['Wind_Direction']))
        df['wind_dir_cos'] = np.cos(np.radians(df['Wind_Direction']))
    return df

# 2. Modify the preprocess_data function to include wind direction encoding
def preprocess_data(df, base_year=2000):
    # Original preprocessing
    # Identify numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # Define outlier thresholds using IQR method
    Q1 = df[numeric_cols].quantile(0.25)
    Q3 = df[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the column's mean value
    for col in numeric_cols:
        mean_value = df[col].mean()
        df[col] = np.where((df[col] < lower_bound[col]) | (df[col] > upper_bound[col]), mean_value, df[col])

    # Convert temperatures > 100 from Kelvin to Celsius
    for temp_col in ['Avg_Temperature', 'Avg_Feels_Like_Temperature']:
        if temp_col in df.columns:
            df[temp_col] = df[temp_col].apply(lambda x: x - 273.15 if x > 100 else x)

    # Create date feature if date columns exist
    date_cols = ["Year", "Month", "Day"]
    if all(col in df.columns for col in date_cols):
        df['Date'] = pd.to_datetime(
            (df['Year'] + base_year).astype(str) + '-' +
            df['Month'].astype(str) + '-' +
            df['Day'].astype(str),
            format='%Y-%m-%d',
            errors='coerce'
        )

        # Add cyclical features to capture seasonality
        df = add_cyclical_features(df)

    # Add circular encoding for wind direction
    df = encode_wind_direction(df)

    return df

# 3. Add this function to convert sine/cosine components back to degrees
def wind_direction_from_components(sin_val, cos_val):
    """Convert sine and cosine components back to degrees"""
    degrees = np.degrees(np.arctan2(sin_val, cos_val))
    # Ensure result is in the range [0, 360)
    return (degrees + 360) % 360

# 4. Add a circular error metric function for wind direction evaluation
def circular_mae(actual, predicted, period=360):
    """
    Calculate Mean Absolute Error accounting for circular nature of wind direction

    Args:
        actual: Array of actual wind direction values in degrees
        predicted: Array of predicted wind direction values in degrees
        period: The period of the circular variable (360 for wind direction)

    Returns:
        Mean absolute error in degrees, accounting for circularity
    """
    # Convert to radians for calculation
    actual_rad = np.radians(actual)
    predicted_rad = np.radians(predicted)

    # Calculate circular difference
    diff = np.abs(np.arctan2(
        np.sin(predicted_rad - actual_rad),
        np.cos(predicted_rad - actual_rad)
    ))

    # Convert back to degrees and return mean
    return np.degrees(diff).mean()

def process_locations(df):
    # Use LabelEncoder for the location names
    label_encoder = LabelEncoder()
    location_encoded = label_encoder.fit_transform(df['kingdom'])
    # Convert to a numpy array and reshape for the model
    location_encoded = np.array(location_encoded).reshape(-1, 1)
    # Get the vocabulary size for the embedding layer
    vocab_size = len(label_encoder.classes_)
    return location_encoded, vocab_size, label_encoder

# Modified normalize_features function to include cyclical features
def normalize_features(df):
    # Select numerical features (excluding categorical and ID columns)
    # Include the new cyclical features
    numerical_cols = [col for col in df.columns if col not in
                     ['kingdom', 'ID', 'Date', 'Year', 'Month', 'Day']
                     and col in df.columns]

    # MinMaxScaler instance
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit and transform the numerical data
    numerical_data = scaler.fit_transform(df[numerical_cols])
    # Return normalized data and the scaler
    return numerical_data, scaler, numerical_cols

# Function to find seasonally matching data
def get_seasonal_matching_data(train_df, current_month, current_day, kingdom, n_steps_in):
    """
    Get data from training set that matches the season of the current prediction
    """
    # Month range for seasonal matching (same month ±1)
    month_lower = max(1, current_month - 1)
    month_upper = min(12, current_month + 1)

    # Filter by kingdom and season
    seasonal_data = train_df[(train_df['kingdom'] == kingdom) &
                            (train_df['Month'] >= month_lower) &
                            (train_df['Month'] <= month_upper)]

    # If not enough data, broaden the month range
    if len(seasonal_data) < n_steps_in:
        month_lower = max(1, current_month - 2)
        month_upper = min(12, current_month + 2)
        seasonal_data = train_df[(train_df['kingdom'] == kingdom) &
                                (train_df['Month'] >= month_lower) &
                                (train_df['Month'] <= month_upper)]

    # If still not enough data, use all data for this kingdom
    if len(seasonal_data) < n_steps_in:
        seasonal_data = train_df[train_df['kingdom'] == kingdom]

    # If still not enough, use all training data
    if len(seasonal_data) < n_steps_in:
        seasonal_data = train_df

    return seasonal_data

def split_data(location_encoded, numerical_data, test_size=0.1, val_size=0.1):
    # Get total sample count
    total_samples = len(numerical_data)
    # Calculate indices for train/val/test splits
    test_start_idx = int(total_samples * (1 - test_size))
    val_start_idx = int(total_samples * (1 - test_size - val_size))
    # Split data chronologically
    X_train = numerical_data[:val_start_idx]
    X_val = numerical_data[val_start_idx:test_start_idx]
    X_test = numerical_data[test_start_idx:]
    loc_train = location_encoded[:val_start_idx]
    loc_val = location_encoded[val_start_idx:test_start_idx]
    loc_test = location_encoded[test_start_idx:]

    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    print(f"Test samples: {len(X_test)}")

    return X_train, X_val, X_test, loc_train, loc_val, loc_test

def prepare_sequences(X_train, X_val, X_test, loc_train, loc_val, loc_test, n_steps_in, n_steps_out):
    # Create sequences for train, validation, and test data
    X_train_seq, y_train_seq = create_sequences(X_train, n_steps_in, n_steps_out)
    loc_train_seq, _ = create_sequences(loc_train, n_steps_in, n_steps_out)

    X_val_seq, y_val_seq = create_sequences(X_val, n_steps_in, n_steps_out)
    loc_val_seq, _ = create_sequences(loc_val, n_steps_in, n_steps_out)

    X_test_seq, y_test_seq = create_sequences(X_test, n_steps_in, n_steps_out)
    loc_test_seq, _ = create_sequences(loc_test, n_steps_in, n_steps_out)

    return (X_train_seq, y_train_seq, loc_train_seq,
            X_val_seq, y_val_seq, loc_val_seq,
            X_test_seq, y_test_seq, loc_test_seq)

In [ ]:
# Model functions
def build_seq2seq_model(n_steps_in, n_steps_out, n_features, vocab_size, embedding_dim=16):
    # Define inputs for features and location
    feature_input = Input(shape=(n_steps_in, n_features), name='feature_input')
    location_input = Input(shape=(n_steps_in, 1), name='location_input')

    # Process location data with embedding
    loc_reshaped = Reshape((-1,))(location_input)
    loc_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(loc_reshaped)
    loc_embedded = Reshape((n_steps_in, embedding_dim))(loc_embedding)

    # Combine features with location embeddings
    combined = Concatenate(axis=2)([feature_input, loc_embedded])

    # Encoder-decoder architecture
    encoder = LSTM(128, activation='relu')(combined)
    repeat = RepeatVector(n_steps_out)(encoder)
    decoder = LSTM(128, activation='relu', return_sequences=True)(repeat)
    output = TimeDistributed(Dense(n_features))(decoder)

    # Create and compile model
    model = Model(inputs=[feature_input, location_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')

    return model

def train_model(model, X_train_seq, y_train_seq, loc_train_seq, X_val_seq, y_val_seq, loc_val_seq, epochs=100, batch_size=64):
    # Train with early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        [X_train_seq, loc_train_seq], y_train_seq,
        validation_data=([X_val_seq, loc_val_seq], y_val_seq),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )

    return history

In [ ]:
# 5. Modified evaluate_model function to handle circular metrics for wind direction
def evaluate_model(model, X_test_seq, y_test_seq, loc_test_seq, history, numerical_cols, scaler):
    # Make predictions
    y_pred = model.predict([X_test_seq, loc_test_seq])

    # Calculate overall metrics on normalized values
    rmse_overall = math.sqrt(mean_squared_error(y_test_seq.reshape(-1), y_pred.reshape(-1)))
    mae_overall = mean_absolute_error(y_test_seq.reshape(-1), y_pred.reshape(-1))

    # Inverse transform predictions and actual values to original scale
    y_test_original = np.zeros_like(y_test_seq)
    y_pred_original = np.zeros_like(y_pred)

    for i in range(len(y_test_seq)):
        y_test_original[i] = scaler.inverse_transform(y_test_seq[i])
        y_pred_original[i] = scaler.inverse_transform(y_pred[i])

    # Calculate per-feature metrics on original scale
    rmse_values = []
    mae_values = []

    # Check if we need to handle wind direction components
    wind_dir_sin_idx = -1
    wind_dir_cos_idx = -1

    if 'wind_dir_sin' in numerical_cols and 'wind_dir_cos' in numerical_cols:
        wind_dir_sin_idx = numerical_cols.index('wind_dir_sin')
        wind_dir_cos_idx = numerical_cols.index('wind_dir_cos')

    # Special handling for wind direction if we have sine and cosine components
    if wind_dir_sin_idx >= 0 and wind_dir_cos_idx >= 0:
        # Convert sine and cosine components back to degrees
        actual_wind_dir = []
        pred_wind_dir = []

        for i in range(y_test_original.shape[0]):
            for j in range(y_test_original.shape[1]):
                # Convert actual values
                actual_sin = y_test_original[i, j, wind_dir_sin_idx]
                actual_cos = y_test_original[i, j, wind_dir_cos_idx]
                actual_deg = wind_direction_from_components(actual_sin, actual_cos)
                actual_wind_dir.append(actual_deg)

                # Convert predicted values
                pred_sin = y_pred_original[i, j, wind_dir_sin_idx]
                pred_cos = y_pred_original[i, j, wind_dir_cos_idx]
                pred_deg = wind_direction_from_components(pred_sin, pred_cos)
                pred_wind_dir.append(pred_deg)

        # Calculate circular error metrics
        wind_dir_circular_mae = circular_mae(np.array(actual_wind_dir), np.array(pred_wind_dir))
        print(f"Wind Direction Circular MAE: {wind_dir_circular_mae:.4f} degrees")

    # Calculate standard metrics for all features
    for i in range(y_test_seq.shape[2]):
        # Skip individual sin/cos components if we already calculated circular metrics
        if i == wind_dir_sin_idx or i == wind_dir_cos_idx:
            continue

        rmse = math.sqrt(mean_squared_error(y_test_original[:,:,i].reshape(-1), y_pred_original[:,:,i].reshape(-1)))
        mae = mean_absolute_error(y_test_original[:,:,i].reshape(-1), y_pred_original[:,:,i].reshape(-1))
        rmse_values.append(rmse)
        mae_values.append(mae)
        print(f"Feature {numerical_cols[i]}: RMSE={rmse:.4f}, MAE={mae:.4f}")

    print(f"Overall Normalized Test RMSE: {rmse_overall:.4f}")
    print(f"Overall Normalized Test MAE: {mae_overall:.4f}")

    # Visualize results with original scale values
    visualize_results(history, y_test_original, y_pred_original, rmse_values, mae_values, numerical_cols)

    return rmse_overall, mae_overall, y_pred_original, rmse_values, mae_values

In [ ]:
def visualize_results(history, y_test_original, y_pred_original, rmse_values, mae_values, feature_names):
    plt.figure(figsize=(16, 12))

    # Plot loss
    plt.subplot(3, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    # Plot temperature prediction for a sample (in original scale)
    plt.subplot(3, 2, 2)
    temp_idx = feature_names.index('Avg_Temperature') if 'Avg_Temperature' in feature_names else 0
    plt.plot(y_test_original[0, :, temp_idx], label='Actual')
    plt.plot(y_pred_original[0, :, temp_idx], label='Predicted')
    plt.title('Prediction vs Actual (Temperature °C)')
    plt.xlabel('Time Step')
    plt.ylabel('Temperature (°C)')
    plt.legend()

    # Plot feature-wise RMSE
    plt.subplot(3, 2, 3)
    # Ensure we're only plotting features that have RMSE values calculated
    plot_features = feature_names[:len(rmse_values)]
    x_pos = np.arange(len(rmse_values))
    plt.bar(x_pos, rmse_values)
    plt.xticks(x_pos, plot_features, rotation=90)
    plt.title('RMSE by Feature (Original Scale)')
    plt.ylabel('RMSE')

    # Plot feature-wise MAE
    plt.subplot(3, 2, 4)
    plt.bar(x_pos, mae_values)
    plt.xticks(x_pos, plot_features, rotation=90)
    plt.title('MAE by Feature (Original Scale)')
    plt.ylabel('MAE')

    # Plot additional predictions for important features
    if len(feature_names) > 1:
        # Plot radiation prediction
        rad_idx = feature_names.index('Radiation') if 'Radiation' in feature_names else 1
        if rad_idx < y_test_original.shape[2]:
            plt.subplot(3, 2, 5)
            plt.plot(y_test_original[0, :, rad_idx], label='Actual')
            plt.plot(y_pred_original[0, :, rad_idx], label='Predicted')
            plt.title('Prediction vs Actual (Radiation W/m²)')
            plt.xlabel('Time Step')
            plt.ylabel('Radiation (W/m²)')
            plt.legend()

        # Plot wind speed prediction
        wind_idx = feature_names.index('Wind_Speed') if 'Wind_Speed' in feature_names else 2
        if wind_idx < y_test_original.shape[2]:
            plt.subplot(3, 2, 6)
            plt.plot(y_test_original[0, :, wind_idx], label='Actual')
            plt.plot(y_pred_original[0, :, wind_idx], label='Predicted')
            plt.title('Prediction vs Actual (Wind Speed km/h)')
            plt.xlabel('Time Step')
            plt.ylabel('Wind Speed (km/h)')
            plt.legend()

    plt.tight_layout()
    plt.savefig('prediction_results.png')
    plt.show()

In [ ]:
# Modified run_pipeline function to include monitoring of prediction drift
def run_pipeline(df, n_steps_in=30, n_steps_out=7, epochs=50, batch_size=64):
    # Process data
    # (Using df that already has cyclical features added in preprocess_data)
    location_encoded, vocab_size, label_encoder = process_locations(df)
    numerical_data, scaler, numerical_cols = normalize_features(df)

    # Print numerical columns to verify cyclical features are included
    print(f"Features used in model: {numerical_cols}")

    # Split data
    X_train, X_val, X_test, loc_train, loc_val, loc_test = split_data(location_encoded, numerical_data)

    # Prepare sequences
    seq_data = prepare_sequences(
        X_train, X_val, X_test, loc_train, loc_val, loc_test, n_steps_in, n_steps_out
    )
    X_train_seq, y_train_seq, loc_train_seq, X_val_seq, y_val_seq, loc_val_seq, X_test_seq, y_test_seq, loc_test_seq = seq_data

    # Build model
    model = build_seq2seq_model(
        n_steps_in=n_steps_in,
        n_steps_out=n_steps_out,
        n_features=X_train.shape[1],
        vocab_size=vocab_size
    )
    model.summary()

    # Train model
    history = train_model(
        model,
        X_train_seq, y_train_seq, loc_train_seq,
        X_val_seq, y_val_seq, loc_val_seq,
        epochs=epochs,
        batch_size=batch_size
    )

    # Evaluate model with original scale values
    metrics = evaluate_model(
        model,
        X_test_seq, y_test_seq, loc_test_seq,
        history,
        numerical_cols,
        scaler
    )
    rmse, mae, y_pred, rmse_values, mae_values = metrics

    # Return results
    return {
        'model': model,
        'label_encoder': label_encoder,
        'scaler': scaler,
        'numerical_cols': numerical_cols,
        'rmse': rmse,
        'mae': mae,
        'rmse_by_feature': rmse_values,
        'mae_by_feature': mae_values,
        'history': history,
        'y_pred': y_pred,
        'y_test': y_test_seq
    }

In [ ]:
def predict_on_test(test_df, train_df, n_steps_in=30, n_steps_out=7, target_cols=None, reset_window_days=7):
    """
    Make predictions on test.csv using the trained model with sliding window approach
    and seasonal pattern incorporation

    Args:
        test_df: DataFrame containing test data
        train_df: DataFrame containing training data
        n_steps_in: Number of input time steps
        n_steps_out: Number of output time steps
        target_cols: List of target columns to predict
        reset_window_days: How often to reset the prediction window with seasonal training data

    Returns:
        DataFrame with predictions for all test rows
    """
    # Default target columns if none provided
    if target_cols is None:
        target_cols = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']

    # Print information about the data
    print(f"Total rows in test data: {len(test_df)}")
    print(f"Number of unique kingdoms: {test_df['kingdom'].nunique()}")

    # Preprocess training data to include cyclical features and wind direction encoding
    train_df = preprocess_data(train_df)

    # Keep track of whether Wind_Direction is among target columns
    needs_wind_direction = 'Wind_Direction' in target_cols

    # Train the model with enhanced features
    results = run_pipeline(train_df, n_steps_in=n_steps_in, n_steps_out=n_steps_out, epochs=50)
    model = results['model']
    label_encoder = results['label_encoder']
    scaler = results['scaler']
    numerical_cols = results['numerical_cols']

    # Find indices of wind direction components in numerical cols
    wind_dir_sin_idx = -1
    wind_dir_cos_idx = -1
    if 'wind_dir_sin' in numerical_cols and 'wind_dir_cos' in numerical_cols:
        wind_dir_sin_idx = numerical_cols.index('wind_dir_sin')
        wind_dir_cos_idx = numerical_cols.index('wind_dir_cos')

    # Preprocess test data including cyclical features and wind direction encoding
    test_df = preprocess_data(test_df)

    # Initialize results DataFrame with all test data
    result_df = test_df[['ID']].copy()

    # Make sure target columns exist in result_df
    for col in target_cols:
        if col not in result_df.columns:
            result_df[col] = np.nan

    # Track prediction coverage
    total_predictions = 0

    # Group by kingdom to handle each location separately
    # Sort first by kingdom, then by date to ensure chronological order
    test_df = test_df.sort_values(by=['kingdom', 'Year', 'Month', 'Day'])
    grouped = test_df.groupby('kingdom')

    for kingdom, group in grouped:
        print(f"Processing kingdom: {kingdom}, rows: {len(group)}")

        # Get training data for this kingdom
        train_kingdom = train_df[train_df['kingdom'] == kingdom].sort_values(by=['Year', 'Month', 'Day'])

        # If kingdom not in training data, use average from all training data
        if len(train_kingdom) == 0:
            print(f"Warning: No training data for {kingdom}. Using global averages.")
            train_kingdom = train_df.copy()

        # Encode location
        try:
            location_encoded = label_encoder.transform([kingdom])[0]
        except ValueError:
            print(f"Warning: {kingdom} not seen during training. Using a default kingdom.")
            # Use the first kingdom in the training data
            default_kingdom = label_encoder.classes_[0]
            location_encoded = label_encoder.transform([default_kingdom])[0]

        # Track days since last window reset
        days_since_reset = reset_window_days  # Start with a reset

        # Create initial historical data from seasonally appropriate training data
        first_row = group.iloc[0]
        current_month = first_row['Month']
        current_day = first_row['Day']

        # Get seasonal data for initial window
        seasonal_data = get_seasonal_matching_data(
            train_df, current_month, current_day, kingdom, n_steps_in
        )

        if len(seasonal_data) >= n_steps_in:
            # Use the most recent n_steps_in days from seasonal data
            initial_history = seasonal_data.sort_values(
                by=['Year', 'Month', 'Day']
            ).tail(n_steps_in)[numerical_cols]
        else:
            # Use what we have and pad with means
            initial_history = seasonal_data[numerical_cols]
            # Calculate mean values for each feature
            mean_values = train_df[numerical_cols].mean()
            # Create padding with mean values
            padding_rows = n_steps_in - len(initial_history)
            padding = pd.DataFrame([mean_values] * padding_rows)
            initial_history = pd.concat([padding, initial_history], ignore_index=True)

        # Convert to numpy array
        history = initial_history.values

        # Store previous predictions for RMSE calculation and monitoring
        previous_predictions = []
        actual_values = []

        # For each day in the test set for this kingdom, predict the values
        for i in range(len(group)):
            current_row = group.iloc[i]
            current_id = current_row['ID']
            current_month = current_row['Month']
            current_day = current_row['Day']

            # Check if we need to reset the window with fresh seasonal data
            if days_since_reset >= reset_window_days:
                print(f"Resetting prediction window for {kingdom} at month {current_month}, day {current_day}")

                # Get seasonal matching data for current time
                seasonal_data = get_seasonal_matching_data(
                    train_df, current_month, current_day, kingdom, n_steps_in
                )

                if len(seasonal_data) >= n_steps_in:
                    # Reset history with seasonal data
                    seasonal_history = seasonal_data.sort_values(
                        by=['Year', 'Month', 'Day']
                    ).tail(n_steps_in)[numerical_cols].values

                    # Blend current history with seasonal history (gradual transition)
                    blend_factor = 0.7  # 70% seasonal, 30% current trajectory
                    history = blend_factor * seasonal_history + (1 - blend_factor) * history

                    # Reset counter
                    days_since_reset = 0

            # Create location input (same location for all time steps)
            location_seq = np.full((1, n_steps_in, 1), location_encoded)

            # Normalize features
            history_df = pd.DataFrame(history, columns=numerical_cols)
            feature_input_normalized = scaler.transform(history_df).reshape(1, n_steps_in, -1)

            # Make prediction
            prediction = model.predict([feature_input_normalized, location_seq], verbose=0)

            # We're only interested in the first prediction (next day)
            next_day_pred = prediction[0, 0, :]

            # Convert back to original scale
            next_day_pred_df = pd.DataFrame(next_day_pred.reshape(1, -1), columns=numerical_cols)
            next_day_pred_original = scaler.inverse_transform(next_day_pred_df)

            # Store prediction for monitoring
            previous_predictions.append(next_day_pred_original[0])

            # Handle Wind_Direction prediction from sine and cosine components if needed
            if needs_wind_direction and wind_dir_sin_idx >= 0 and wind_dir_cos_idx >= 0:
                # Extract sine and cosine components
                wind_dir_sin = next_day_pred_original[0, wind_dir_sin_idx]
                wind_dir_cos = next_day_pred_original[0, wind_dir_cos_idx]

                # Convert back to degrees
                wind_dir_degrees = wind_direction_from_components(wind_dir_sin, wind_dir_cos)

                # Add to result DataFrame
                result_df.loc[result_df['ID'] == current_id, 'Wind_Direction'] = wind_dir_degrees

            # Update result DataFrame with predictions for other columns
            for j, col in enumerate(numerical_cols):
                # Skip wind_dir_sin and wind_dir_cos as we handle them separately above
                if col in ['wind_dir_sin', 'wind_dir_cos']:
                    continue

                if col in target_cols:
                    result_df.loc[result_df['ID'] == current_id, col] = next_day_pred_original[0, j]

            # Count this prediction
            total_predictions += 1

            # Prepare for next day prediction by updating history
            # Use actual values from test set where available
            actual_values_for_next = []
            for col_idx, col in enumerate(numerical_cols):
                if col in ['wind_dir_sin', 'wind_dir_cos'] and 'Wind_Direction' in current_row and not pd.isna(current_row['Wind_Direction']):
                    # Convert actual Wind_Direction to sine and cosine
                    actual_wind_dir = current_row['Wind_Direction']
                    if col == 'wind_dir_sin':
                        actual_values_for_next.append(np.sin(np.radians(actual_wind_dir)))
                    else:  # col == 'wind_dir_cos'
                        actual_values_for_next.append(np.cos(np.radians(actual_wind_dir)))

                    # Store for monitoring
                    if col == 'wind_dir_sin' and needs_wind_direction:
                        # Calculate the corresponding predicted Wind_Direction value
                        wind_dir_sin = next_day_pred_original[0, wind_dir_sin_idx]
                        wind_dir_cos = next_day_pred_original[0, wind_dir_cos_idx]
                        predicted_wind_dir = wind_direction_from_components(wind_dir_sin, wind_dir_cos)
                        actual_values.append(('Wind_Direction', actual_wind_dir, predicted_wind_dir))

                elif col in current_row and not pd.isna(current_row[col]):
                    # Use actual value from test data
                    actual_values_for_next.append(current_row[col])

                    # Store for monitoring if this is a target column
                    if col in target_cols:
                        actual_values.append((col, current_row[col], next_day_pred_original[0, col_idx]))
                elif col in target_cols:
                    # Use our prediction
                    actual_values_for_next.append(next_day_pred_original[0, col_idx])
                else:
                    # Use the mean value from training
                    actual_values_for_next.append(train_df[col].mean())

            # Roll the history window forward
            history = np.vstack([history[1:], actual_values_for_next])

            # Increment days since last reset
            days_since_reset += 1

        # After processing kingdom, calculate and print RMSE for monitoring
        if actual_values:
            # Convert to DataFrame for easier analysis
            actual_df = pd.DataFrame(actual_values, columns=['feature', 'actual', 'predicted'])

            # Calculate RMSE by feature
            for feature in actual_df['feature'].unique():
                feature_df = actual_df[actual_df['feature'] == feature]
                if feature == 'Wind_Direction':
                    # Use circular metric for wind direction
                    mae = circular_mae(feature_df['actual'], feature_df['predicted'])
                    print(f"Kingdom {kingdom}, Feature {feature}: Circular MAE = {mae:.4f}")
                else:
                    # Use standard RMSE for other features
                    rmse = np.sqrt(mean_squared_error(feature_df['actual'], feature_df['predicted']))
                    print(f"Kingdom {kingdom}, Feature {feature}: RMSE = {rmse:.4f}")

    # Verify we have predictions for all rows
    print(f"Made predictions for {total_predictions} of {len(test_df)} test rows")

    # Check how many rows have NaN values in target columns
    nan_count = result_df[target_cols].isna().any(axis=1).sum()
    if nan_count > 0:
        print(f"Warning: {nan_count} rows have NaN values in target columns")

    # Keep only ID and target columns in the final output
    output_cols = ['ID'] + [col for col in target_cols if col in result_df.columns]
    result_df = result_df[output_cols]

    # Save predictions to CSV
    result_df.to_csv("drive/MyDrive/DataCrunch/predictions.csv", index=False)
    print(f"Predictions saved to predictions.csv with {len(result_df)} rows")

    return result_df

# Function to convert sine/cosine components back to degrees
def wind_direction_from_components(sin_val, cos_val):
    """Convert sine and cosine components back to degrees"""
    degrees = np.degrees(np.arctan2(sin_val, cos_val))
    # Ensure result is in the range [0, 360)
    return (degrees + 360) % 360

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Main execution function
if __name__ == "__main__":
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Load training data
    train_df = pd.read_csv("drive/MyDrive/DataCrunch/train.csv")

    # Load test data
    test_df = pd.read_csv("drive/MyDrive/DataCrunch/test.csv")

    # Specify target columns to predict
    target_columns = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']

    # Make predictions on test data with sliding window approach
    # Reset window every 5 days to prevent drift
    predictions = predict_on_test(
        test_df=test_df,
        train_df=train_df,
        n_steps_in=30,
        n_steps_out=7,
        target_cols=target_columns,
        reset_window_days=5  # Reset prediction window every 5 days
    )

    print("Prediction process completed")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total rows in test data: 4530
Number of unique kingdoms: 30
Features used in model: ['latitude', 'longitude', 'Avg_Temperature', 'Avg_Feels_Like_Temperature', 'Temperature_Range', 'Feels_Like_Temperature_Range', 'Radiation', 'Rain_Amount', 'Rain_Duration', 'Wind_Speed', 'Wind_Direction', 'Evapotranspiration', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'wind_dir_sin', 'wind_dir_cos']
Training samples: 67968
Validation samples: 8496
Test samples: 8496


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ location_input            │ (None, 30, 1)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 30)             │              0 │ location_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 30, 16)         │            480 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ feature_input             │ (None, 30, 18)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 30, 16)         │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 30, 34)         │              0 │ feature_input[0][0],   │
│                           │                        │                │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 128)            │         83,456 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_vector             │ (None, 7, 128)         │              0 │ lstm[0][0]             │
│ (RepeatVector)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 7, 128)         │        131,584 │ repeat_vector[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 7, 18)          │          2,322 │ lstm_1[0][0]           │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 217,842 (850.95 KB)

 Trainable params: 217,842 (850.95 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 115s 102ms/step - loss: 0.0420 - val_loss: 0.0201
Epoch 2/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 143s 104ms/step - loss: 0.0185 - val_loss: 0.0161
Epoch 3/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 143s 105ms/step - loss: 0.0137 - val_loss: 0.0125
Epoch 4/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 117s 110ms/step - loss: 0.0108 - val_loss: 0.0113
Epoch 5/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 139s 107ms/step - loss: 0.0095 - val_loss: 0.0103
Epoch 6/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 142s 108ms/step - loss: 0.0086 - val_loss: 0.0103
Epoch 7/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 139s 105ms/step - loss: 0.0082 - val_loss: 0.0095
Epoch 8/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 146s 109ms/step - loss: 0.0077 - val_loss: 0.0092
Epoch 9/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 107s 101ms/step - loss: 0.0074 - val_loss: 0.0089
Epoch 10/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 156s 147ms/step - loss: 0.0072 - val_loss: 0.0090
Epoch 11/50
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 153s 101ms/step - loss: 0.0070 - val